In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.<enter version>'
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3-us-west-1.amazonaws.com/big.data.dd/amazon_reviews_us_Jewelry_v1_00.tsv"
spark.sparkContext.addFile(url)

df = spark.read.format('csv').option('inferSchema', 'true').option('header', 'true').option('delimiter', '\t').csv(SparkFiles.get('amazon_reviews_us_Jewelry_v1_00.tsv'))
df.show(10)

In [ ]:
# Count rows
df.count()

In [ ]:
# Print and examine schema
df.printSchema()

In [ ]:
# Jewelry dataframe
df.createOrReplaceTempView('jewelry')

In [ ]:
# Reviews dataframe
review_id_table = spark.sql("""SELECT review_id, customer_id, product_id, product_parent, to_date(review_date) as review_date FROM jewelry


""").show(10)

In [ ]:
# Products dataframe
products = spark.sql("""SELECT product_id, product_title FROM jewelry


""").drop_duplicates()
products.show(10)

In [ ]:
# Customers dataframe
customers = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers.show(10)

In [ ]:
# Vine dataframe
vine_table = spark.sql("""SELECT review_id, star_rating, helpful_votes, total_votes, vine FROM jewelry


""").show(10)

In [ ]:
# Configuration for RDS instance

In [ ]:
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
config = {"user":"root",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write dataframes to tables

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)